In [ ]:
import pandas as pd
import quandl
import math

import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [14]:
# getting data for GOOGL stock from quandl

df = quandl.get('WIKI/GOOGL')
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio',
       'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'],
      dtype='object')

In [13]:
# quandl.ApiConfig.api_key = 'tCYts9m9cqo4yDkwyAym'
# df = quandl.get('BCHARTS/LOCALBTCINR')
# df.tail()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2019-01-04,283000.02,3333600.02,250499.21,286154.80,34.600636,9.710059e+06,280632.391138
2019-01-05,295500.06,2222222.22,260861.91,266925.20,36.729558,1.032165e+07,281017.626914
2019-01-06,274000.00,2333286.67,238099.50,305058.48,24.702179,6.990754e+06,283001.519654
2019-01-07,296900.01,2222222.22,260000.00,304673.69,39.296291,1.144308e+07,291200.111587
2019-01-08,286101.03,389153.51,272407.01,294498.76,16.707677,4.797441e+06,287139.883396


In [17]:
# picking relevant columns from the downloaded dataframe

df = df[['Adj. High','Adj. Low','Adj. Open','Adj. Close', 'Adj. Volume']]

# adding a metric: High-Low Percent
df ['HL_PCT'] = ((df['Adj. High'] - df['Adj. Low']) / df['Adj. Low']) * 100

# adding a metric: Percent Change
df ['CHANGE_PCT'] = ((df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open']) * 100

In [ ]:
# discarding colums that have been used for the metrics and are no longer needed.
df = df[['Adj. Close', 'HL_PCT', 'CHANGE_PCT', 'Adj. Volume']]


# The column that we wan't our model to predict/forcast
forecast_column = 'Adj. Close'

# Data Cleaning : replacing all "na" values
df.fillna('-99999', inplace=True)

In [24]:
# How many enteries ahead do we wan't to forecast
# forecast_out = int(math.ceil(0.1 * len(df)))
forecast_out = 10

# we create a label column that is actually the forecast_column but forecast_out days into the future.
df['label'] = df[forecast_column].shift(-forecast_out)

df[['Adj. Close',"label"]].tail(15)

,Adj. Close,label
Date,,
2018-03-07,1115.04,1094.00
2018-03-08,1129.38,1053.15
2018-03-09,1160.84,1026.55
2018-03-12,1165.93,1054.09
2018-03-13,1139.91,1006.94
2018-03-14,1148.89,NaN
2018-03-15,1150.61,NaN
2018-03-16,1134.42,NaN
2018-03-19,1100.07,NaN


In [25]:
# now since we added labels as a shift, we will have rows that have no labels. These will be the last forecast_out rows.
# here we are dropping these rows. These rows ain't loyal.
df.dropna(inplace=True)

# if we didn't drop the rows without labels earlier using dropna, this line could be used to do that.
# X = X[:-forecast_out + 1]

df[['Adj. Close',"label"]].tail(15)

,Adj. Close,label
Date,,
2018-02-21,1113.75,1115.04
2018-02-22,1109.90,1129.38
2018-02-23,1128.09,1160.84
2018-02-26,1143.70,1165.93
2018-02-27,1117.51,1139.91
2018-02-28,1103.92,1148.89
2018-03-01,1071.41,1150.61
2018-03-02,1084.14,1134.42
2018-03-05,1094.76,1100.07


In [26]:
# print(forecast_out)

X = np.array(df.drop(['label'], 1))
y = np.array(df['label'])

X = preprocessing.scale(X)


# print(len(X), len(y))

# now we will split out data into training and testing data.
# test_size defines what portion of the data we want to use for testing,
# the train-test-split function randomly will pick out test_size portion of entries.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)

clf = LinearRegression()
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test);

print(accuracy)

0.9919398709017725
